In [11]:
from typing import List
import pandas as pd
import json
import os

def compute_validation_effectiveness(baseline_file: str, model_names: List, result_dir: str, index_name: str):

    df_baseline = pd.read_csv(baseline_file)
    
    results = []

    for model_name in model_names:
        
        results_file = f"{result_dir}/all_dependencies_{index_name}_{model_name}.json"

        with open(results_file, "r", encoding="utf-8") as src:
            data = json.load(src)

        true_positives = []
        true_negatives = []
        false_positives = []
        false_negatives = []
        accuracy_count = []
        skipped = 0

        for entry, (index, row) in zip(data, df_baseline.iterrows()):

            response_dict = None
            isDependency = None

            rating = row["final_rating"]
            try:
                if "responses" in entry:
                    response = entry["responses"][0]
                else:
                    response = entry["response"]
                response_dict = json.loads(response, strict=False)
                isDependency = response_dict["isDependency"]
            except (json.JSONDecodeError, KeyError):
                skipped += 1
                continue
                
            if str(rating) == "Borderline":
                if isDependency:
                    accuracy_count.append(1)
                    true_positives.append(1)
                if not isDependency:
                    accuracy_count.append(1)
                    true_negatives.append(1)

            # TP: The LLM validates a dependency as correct and the dependency is correct
            if isDependency and str(rating).lower() == "true":
                accuracy_count.append(1)
                true_positives.append(1)
                
            # FP: The LLM validates a dependency as correct, but the dependency is actually incorrect
            if isDependency and str(rating).lower() == "false":
                accuracy_count.append(0)
                false_positives.append(1)

            # TN: The LLM validates a dependency as incorrect and the dependency is incorrect
            if not isDependency and str(rating).lower() == "false":
                accuracy_count.append(1)
                true_negatives.append(1)

            # FN: The LLM validates a dependency as incorrect, but the dependency is actually correct
            if not isDependency and  str(rating).lower() == "true":
                accuracy_count.append(0)
                false_negatives.append(1)

        
        tp = sum(true_positives)
        fp = sum(false_positives)
        fn = sum(false_negatives)
        tn = sum(true_negatives)
        accuracy = sum(accuracy_count)/len(accuracy_count)


        print("Model: ", model_name)
        print("File: ", results_file)
        print("TP", tp)
        print("FP", fp)
        print("TN", tn)
        print("FN", fn)
        print("Sum correct response format: ", sum([tp, fp, tn, fn]))
        print("Entries skipped: ", skipped)

        precision = tp/(tp+fp)
        recall = tp/(tp+fn)
        f1_score = 2 * (precision * recall) / (precision + recall)

        print("Accuracy", accuracy)
        print("Precision", precision)
        print("Recall", recall)
        print("F1 Score: ", f1_score)

        results.append({
            "model_name": model_name,
            "file": results_file,
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1-score": f1_score
        })
    

    df_results = pd.DataFrame(results)
    df_results.to_csv(f"{result_dir}/validation_effectiveness.csv", index=False)


compute_validation_effectiveness(
    baseline_file="../data/results/all_dependencies.csv",
    model_names=["gpt-4o-2024-05-13", "gpt-3.5-turbo-0125", "llama3:70b", "llama3:8b", "llama3.1:8b"], #"llama3.1:70b"
    result_dir="../data/results/config4",
    index_name="all"
)

Model:  gpt-4o-2024-05-13
File:  ../data/results/config4/all_dependencies_all_gpt-4o-2024-05-13.json
TP 115
FP 28
TN 269
FN 88
Sum correct response format:  500
Entries skipped:  0
Accuracy 0.768
Precision 0.8041958041958042
Recall 0.5665024630541872
F1 Score:  0.6647398843930635
Model:  gpt-3.5-turbo-0125
File:  ../data/results/config4/all_dependencies_all_gpt-3.5-turbo-0125.json
TP 144
FP 127
TN 170
FN 59
Sum correct response format:  500
Entries skipped:  0
Accuracy 0.628
Precision 0.5313653136531366
Recall 0.7093596059113301
F1 Score:  0.6075949367088609
Model:  llama3:70b
File:  ../data/results/config4/all_dependencies_all_llama3:70b.json
TP 149
FP 108
TN 189
FN 54
Sum correct response format:  500
Entries skipped:  0
Accuracy 0.676
Precision 0.5797665369649806
Recall 0.7339901477832512
F1 Score:  0.6478260869565218
Model:  llama3:8b
File:  ../data/results/config4/all_dependencies_all_llama3:8b.json
TP 135
FP 140
TN 97
FN 27
Sum correct response format:  399
Entries skipped:  101


In [12]:
import pandas as pd

file_path = "../data/results/config4/validation_effectiveness.csv"

df = pd.read_csv(file_path)

df

,model_name,file,accuracy,precision,recall,f1-score
0,gpt-4o-2024-05-13,../data/results/config4/all_dependencies_all_g...,0.768000,0.804196,0.566502,0.664740
1,gpt-3.5-turbo-0125,../data/results/config4/all_dependencies_all_g...,0.628000,0.531365,0.709360,0.607595
2,llama3:70b,../data/results/config4/all_dependencies_all_l...,0.676000,0.579767,0.733990,0.647826
3,llama3:8b,../data/results/config4/all_dependencies_all_l...,0.581454,0.490909,0.833333,0.617849
4,llama3.1:8b,../data/results/config4/all_dependencies_all_l...,0.616205,0.527950,0.449735,0.485714
